In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import sys
import os
sys.path.append("/Users/chernals/cernbox/Work/packages")
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
### Defining a default context
brho = georges.physics.momentum_to_brho(georges.physics.energy_to_momentum(177.5))
context = {
        'PARTICLE': 'PROTON',
        'PC': georges.physics.energy_to_momentum(177.5),
        'BETAX'   : 0.0846155,
        'BETAY'   : 0.0846155,
        'ALPHAX': 0.0,
        'ALPHAY': 0.0,
        'DELTAP': 0.0,
        'DPP': 0.5e-2,
        'MOMENTUM_SLITS_OPENING' : 1.015,
        'DIVERGENCE_SLITS_OPENING_X' : 1.015,
        'DIVERGENCE_SLITS_OPENING_Y' : 1.015,
        'N_TRACKING': 5000,
        'EMITX': 14.3e-6,
        'EMITY': 14.3e-6,
    }

In [10]:
import georges
PATH = "/Users/chernals/Dropbox/IBA/Work/IBA-Optics/beamlines"
#bdsim = georges.bdsim.BDSim(bdsim='/Users/chernals/bdsim-build/bdsim', path=PATH)
bl = georges.Beamline(['ess'], path=PATH, prefix='generic', elements='elements')
bl.line
georges.madx.survey(line=bl, context=context).line



,KEYWORD,S,L,ANGLE_SURVEY,X_SURVEY,Y_SURVEY,Z,THETA,PHI,PSI,...,E2,HGAP,THICK,PLUG,CHAMBER,AT_CENTER,AT_ENTRY,AT_EXIT,ORBIT_LENGTH,PHYSICAL
ESS$START,MARKER,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,True
Q1E,QUADRUPOLE,0.295000,0.295000,0.000000,0.000000,0,0.295000,0.000000,0,0,...,NaN,NaN,NaN,K1,0.005,0.147500,0.000000,0.295000,0.295000,True
DRIFT_0,DRIFT,0.627997,0.332997,0.000000,0.000000,0,0.627997,0.000000,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,True
Q2E,QUADRUPOLE,0.922997,0.295000,0.000000,0.000000,0,0.922997,0.000000,0,0,...,NaN,NaN,NaN,K1,0.005,0.775497,0.627997,0.922997,0.295000,True
DRIFT_1,DRIFT,1.060997,0.138001,0.000000,0.000000,0,1.060997,0.000000,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,True
Q3E,QUADRUPOLE,1.355997,0.295000,0.000000,0.000000,0,1.355997,0.000000,0,0,...,NaN,NaN,NaN,K1,0.005,1.208497,1.060997,1.355997,0.295000,True
DRIFT_2,DRIFT,2.184206,0.828208,0.000000,0.000000,0,2.184206,0.000000,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,True
B1E,RBEND,2.970200,0.785994,0.523599,-0.201114,0,2.934775,-0.523599,0,0,...,0.0,0.023,True,K0,0.005,2.577203,2.188680,2.965726,0.777046,True
DRIFT_3,DRIFT,3.376506,0.406306,0.000000,-0.404267,0,3.286646,-0.523599,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,True
B2E,RBEND,4.162500,0.785994,0.523599,-0.953722,0,3.836101,-1.047198,0,0,...,0.0,0.023,True,K0,0.005,3.769503,3.380980,4.158026,0.777046,True


In [5]:
bdsim.attach(bl_survey)
bdsim.run({})
#bdsim.print_input()

NameError: name 'madx_syntax' is not defined

In [2]:
brho = georges.physics.momentum_to_brho(georges.physics.energy_to_momentum(177.5))
context = {
        'PARTICLE': 'PROTON',
        'PC': georges.physics.energy_to_momentum(177.5),
        'BETAX'   : 0.0846155,
        'BETAY'   : 0.0846155,
        'ALPHAX': 0.0,
        'ALPHAY': 0.0,
        'DELTAP': 0.0,
        'DPP': 0.5e-2,
        'IQ1E': 2.741/10/brho/0.0325*0.297/0.295,
        'IQ2E': -5.745/10/brho/0.0325*0.297/0.295,
        'IQ3E': 4.079/10/brho/0.0325*0.297/0.295,
        'IQ47E': -3.062/10/brho/0.0325,
        'IQ56E': 2.66/10/brho/0.0325,
        'IQ8E': 3.5484/10/brho/0.0325,
        'IQ9E': -4.0276/10/brho/0.0325,
        'IQ10E': 4.0352/10/brho/0.0325,
        'MOMENTUM_SLITS_OPENING' : 1.015,
        'DIVERGENCE_SLITS_OPENING_X' : 1.015,
        'DIVERGENCE_SLITS_OPENING_Y' : 1.015,
        'N_TRACKING': 5000,
        'EMITX': 14.3e-6,
        'EMITY': 14.3e-6,
    }
bl_survey = georges.madx.survey(line=bl, madx=m)
#bl_twiss= georges.madx.twiss(line=bl, madx=m)
#b = georges.Beam(energy=200).from_5d_multigaussian_distribution(5,
#                                                   XRMS=0.01,
#                                                   PXRMS=0.001,
#                                                   YRMS=0.01,
#                                                   PYRMS=0.001,
#                                                   DPPRMS=0.00)
#bl_track = georges.madx.track(line=bl, madx=m, beam=b, ptc=False)
#from georges.plotting import *
#with plt.style.context('word'):
#    fig = plt.figure(1)

#    ax1 = fig.add_subplot(111)
#    prepare(ax1, bl)
#    aperture(ax1, bl)
    #twiss(ax1, bl_twiss)
    #track(ax1, bl, b)